In [1]:
import torch
import matplotlib.pyplot as plt
import os
import cv2
import matplotlib.lines as lines
import pandas as pd
import time
import numpy as np
import sys
from shapely.geometry import Polygon
import seaborn as sns
%matplotlib inline

In [2]:
weights = 'yolov5/runs/train/yolov5m/weights/last.pt'
model = torch.hub.load('yolov5/', 'custom', weights, source='local')

requirements: pandas>=1.1.4 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pandas>=1.1.4' ' returned non-zero exit status 1.
YOLOv5  v6.1-190-g4d59f65 Python-3.7.8 torch-1.8.1+cu102 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 20856975 parameters, 0 gradients
Adding AutoShape... 


# Model detections

In [3]:
fig, ax = plt.subplots(2,4, figsize=(16,8))
classes = model.names
colors = [(255,0,0), (0,0,255)]
scaleFont = 2
bbox_width = 6

for region, r, r_i in zip(['europe', 'world'], ['Europe', 'Outside Europe'], range(2)):
    for animal, a, a_i in zip(['dog', 'wolf'], ['Dog', 'Wolf'], range(2)):
        images = os.listdir('images/'+region+'_'+animal+'/')
        
        img = cv2.imread('images/'+region+'_'+animal+'/'+images[0])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
        p = model(img)
        for i in range(len(p.tolist()[0].pred[0])):
            bbox = p.tolist()[0].pred[0][i][0:4].data.cpu().numpy().astype(np.int32)
            prob = p.tolist()[0].pred[0][i][4:6].data.cpu().numpy()
            class_id = np.argmax(prob, axis=0)
            if prob[class_id]>0.7:
                img =cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), colors[class_id], bbox_width)
                label = classes[class_id] +" "+"{:.2f}".format(prob[class_id])
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_TRIPLEX , scaleFont, 2)   
                img = cv2.rectangle(img, (bbox[0]-int(bbox_width/2), bbox[1] - h-10), (bbox[0]+w+bbox_width, bbox[1]), colors[class_id], -1)
                img = cv2.putText(img, label, (bbox[0], bbox[1] - 5),cv2.FONT_HERSHEY_TRIPLEX , scaleFont, (255,255,255), 2)
        
        ax[(a_i)][r_i*2].imshow(img)
        ax[(a_i)][r_i*2].axis('off')
        
        img = cv2.imread('images/'+region+'_'+animal+'/'+images[1])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
        p = model(img)
        for i in range(len(p.tolist()[0].pred[0])):
            bbox = p.tolist()[0].pred[0][i][0:4].data.cpu().numpy().astype(np.int32)
            prob = p.tolist()[0].pred[0][i][4:6].data.cpu().numpy()
            class_id = np.argmax(prob, axis=0)
            if prob[class_id]>0.7:
                img =cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), colors[class_id], bbox_width)
                label = classes[class_id] +" "+"{:.2f}".format(prob[class_id])
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_TRIPLEX , scaleFont, 2)   
                img = cv2.rectangle(img, (bbox[0]-int(bbox_width/2), bbox[1] - h-10), (bbox[0]+w+bbox_width, bbox[1]), colors[class_id], -1)
                img = cv2.putText(img, label, (bbox[0], bbox[1] - 5),cv2.FONT_HERSHEY_TRIPLEX , scaleFont, (255,255,255), 2)
        
        ax[(a_i)][(r_i*2)+1].imshow(p.tolist()[0].imgs[0])
        ax[(a_i)][(r_i*2)+1].axis('off')
        

plt.tight_layout() 
plt.savefig('img_detections.png', bbox_inches='tight', dpi=600)
plt.show()

C:\Users\virgi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


# Model results

In [4]:
results = pd.read_csv('yolov5/runs/train/yolov5m/results.csv')
results.head()

,epoch,train/box_loss,train/obj_loss,train/cls_loss,metrics/precision,...,val/obj_loss,val/cls_loss,x/lr0,x/lr1,x/lr2
0,0,0.081556,0.030069,0.025584,0.22817,...,0.017370,0.018699,0.003314,0.003314,0.070174
1,1,0.062958,0.021209,0.019502,0.35055,...,0.011162,0.015543,0.006516,0.006516,0.040043
2,2,0.059621,0.018070,0.015664,0.42735,...,0.011307,0.014821,0.009585,0.009585,0.009779
3,3,0.052795,0.017665,0.015351,0.52542,...,0.009760,0.010179,0.009406,0.009406,0.009406
4,4,0.046919,0.016685,0.012645,0.62448,...,0.009492,0.008518,0.009406,0.009406,0.009406


In [5]:
results.columns

Index(['               epoch', '      train/box_loss', '      train/obj_loss',
       '      train/cls_loss', '   metrics/precision', '      metrics/recall',
       '     metrics/mAP_0.5', 'metrics/mAP_0.5:0.95', '        val/box_loss',
       '        val/obj_loss', '        val/cls_loss', '               x/lr0',
       '               x/lr1', '               x/lr2'],
      dtype='object')

In [6]:
plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(1,3, figsize=(16,4), sharey='row')

ax[0].plot(results['               epoch'], results['      train/box_loss'], label='Train')
ax[0].plot(results['               epoch'], results['        val/box_loss'], label='Test')
ax[0].title.set_text('Bounding Box Loss')

ax[1].plot(results['               epoch'], results['      train/obj_loss'], label='Train')
ax[1].plot(results['               epoch'], results['        val/obj_loss'], label='Test')
ax[1].title.set_text('Object Loss')

ax[2].plot(results['               epoch'], results['      train/cls_loss'], label='Train')
ax[2].plot(results['               epoch'], results['        val/cls_loss'], label='Test')
ax[2].title.set_text('Classification Loss')

plt.legend()
plt.tight_layout()
plt.savefig('img_yolov5m_loss.png', bbox_inches='tight', dpi=600)
plt.show()

C:\Users\virgi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [7]:
plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(1,4, figsize=(16,4))

ax[0].plot(results['               epoch'], results['   metrics/precision'])
ax[0].title.set_text('Precision')
ax[0].set_xticks([0,10,20,30,40,50])

ax[1].plot(results['               epoch'], results['      metrics/recall'])
ax[1].title.set_text('Recall')
ax[1].set_xticks([0,10,20,30,40,50])

ax[2].plot(results['               epoch'], results['     metrics/mAP_0.5'])
ax[2].title.set_text('mAP (PASCAL VOC)')
ax[2].set_xticks([0,10,20,30,40,50])

ax[3].plot(results['               epoch'], results['metrics/mAP_0.5:0.95'])
ax[3].title.set_text('mAP (COCO)')
ax[3].set_xticks([0,10,20,30,40,50])

plt.tight_layout()
plt.savefig('img_yolov5m_metrics.png', bbox_inches='tight', dpi=600)
plt.show()

C:\Users\virgi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


# Confusion matrix

In [8]:
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
    return iou

In [9]:
def confusion_matrix(matrix, df, bbox_pred, img_name):
    for row_id, row in df.iterrows():        
        class_id = int(df.iloc[row_id][0])
        box_2 = [
        [df.iloc[row_id][1] - (df.iloc[row_id][3]/2), df.iloc[row_id][2] - (df.iloc[row_id][4]/2)], 
        [df.iloc[row_id][1] + (df.iloc[row_id][3]/2), df.iloc[row_id][2] - (df.iloc[row_id][4]/2)], 
        [df.iloc[row_id][1] + (df.iloc[row_id][3]/2), df.iloc[row_id][2] + (df.iloc[row_id][4]/2)], 
        [df.iloc[row_id][1] - (df.iloc[row_id][3]/2), df.iloc[row_id][2] + (df.iloc[row_id][4]/2)]]
        used = False

        bigger_iou = 0
        bigger_index = 0
        bigger_class = 0
        for row_id, info in bbox_pred.items():
            if not info['used']:
                box_class = info['class']
                iou = calculate_iou(info['bbox'], box_2)
                if iou>bigger_iou:
                    bigger_iou = iou
                    bigger_index = row_id
                    bigger_class = box_class
        if bigger_iou>=0.5:
            matrix[bigger_class][class_id] += 1
            bbox_pred[bigger_index]['used'] = True
            used = True
        elif bigger_iou > 0:
            matrix[2][class_id] += 1
            bbox_pred[bigger_index]['used'] = True
            used = True
        if not used:
            matrix[2][class_id] += 1

    for row_id, info in bbox_pred.items():
        if not info['used']:
            box_class = info['class']
            matrix[box_class][2] += 1
            bbox_pred[row_id]['used'] = True
    return matrix

In [10]:
train_matrix = np.zeros((3,3)) #row, column
for img_name in os.listdir('datasets/dataset/images/train2017'):
    img = cv2.imread('datasets/dataset/images/train2017/'+img_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
    p = model(img)
    row_count = 0
    bbox_pred = {}
    for i in range(len(p.tolist()[0].pred[0])):
        bbox = p.tolist()[0].xyxyn[0].data.cpu().numpy().astype(np.float32)[i][0:4]        
        prob = p.tolist()[0].pred[0][i][4:6].data.cpu().numpy()
        class_id = np.argmax(prob, axis=0)       
        
        if prob[class_id]>0.5:
            box_1 = [[bbox[0], bbox[1]], [bbox[2], bbox[1]], [bbox[2], bbox[3]], [bbox[0], bbox[3]]]
            bbox_pred[str(row_count)] = {'used': False, 'class':class_id, 'bbox':box_1}
            row_count += 1
    df = pd.read_csv('datasets/dataset/labels/train2017/'+img_name[:-3]+'txt', sep=' ', header=None)
    df.head()
    train_matrix =  confusion_matrix(train_matrix, df, bbox_pred, img_name)
train_matrix

array([[        594,           0,          14],
       [          0,         276,          15],
       [          0,           1,           0]])

In [11]:
val_matrix = np.zeros((3,3)) #row, column
for img_name in os.listdir('datasets/dataset/images/val2017'):
    img = cv2.imread('datasets/dataset/images/val2017/'+img_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
    p = model(img)
    row_count = 0
    bbox_pred = {}
    for i in range(len(p.tolist()[0].pred[0])):
        bbox = p.tolist()[0].xyxyn[0].data.cpu().numpy().astype(np.float32)[i][0:4]        
        prob = p.tolist()[0].pred[0][i][4:6].data.cpu().numpy()
        class_id = np.argmax(prob, axis=0)       
        
        if prob[class_id]>0.5:
            box_1 = [[bbox[0], bbox[1]], [bbox[2], bbox[1]], [bbox[2], bbox[3]], [bbox[0], bbox[3]]]
            bbox_pred[str(row_count)] = {'used': False, 'class':class_id, 'bbox':box_1}
            row_count += 1
    df = pd.read_csv('datasets/dataset/labels/val2017/'+img_name[:-3]+'txt', sep=' ', header=None)
    df.head()
    val_matrix =  confusion_matrix(val_matrix, df, bbox_pred, img_name)
val_matrix

array([[        158,           5,           8],
       [          4,          72,          14],
       [         21,           6,           0]])

In [12]:
#train_matrix[train_matrix == 0] = np.nan
#val_matrix[val_matrix == 0] = np.nan

In [13]:
fig, ax = plt.subplots(1,2, figsize=(16,6))
plt.rcParams.update({'font.size': 16})

sns.heatmap(train_matrix, xticklabels= ['Wolf', 'Dog', 'BG'], yticklabels= ['Wolf', 'Dog', 'BG'], annot=True, fmt='g', cmap='rocket_r', ax=ax[0], linewidths=2)

sns.heatmap(val_matrix, xticklabels= ['Wolf', 'Dog', 'BG'], yticklabels= ['Wolf', 'Dog', 'BG'], annot=True, fmt='g', cmap='rocket_r', ax=ax[1], linewidths=2)

ax[0].title.set_text('Train')
ax[1].title.set_text('Validation')
ax[0].set_xlabel('Real')
ax[0].set_ylabel('Predicted')
ax[1].set_xlabel('Real')
ax[1].set_ylabel('Predicted')

plt.tight_layout()
plt.savefig('img_yolov5m_matrix.png', bbox_inches='tight', dpi=600)
plt.show()

C:\Users\virgi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
